In [ ]:
### for accessing file from google drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/MLlab/oct-mtech/
df =pd.read_csv("diabetes.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MLlab/oct-mtech


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# How many missingvalues are mising in each feature
feature_columns = ['Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'
]
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing values : {len(df.loc[df[column] == 0])}")

Glucose ==> Missing values : 5
BloodPressure ==> Missing values : 35
SkinThickness ==> Missing values : 227
Insulin ==> Missing values : 374
BMI ==> Missing values : 11
DiabetesPedigreeFunction ==> Missing values : 0
Age ==> Missing values : 0


In [ ]:
from sklearn.impute import SimpleImputer
fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)
df[feature_columns] = fill_values.fit_transform(df[feature_columns])

for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing values : {len(df.loc[df[column] == 0])}")

Glucose ==> Missing values : 0
BloodPressure ==> Missing values : 0
SkinThickness ==> Missing values : 0
Insulin ==> Missing values : 0
BMI ==> Missing values : 0
DiabetesPedigreeFunction ==> Missing values : 0
Age ==> Missing values : 0


In [ ]:
df.shape

(768, 9)

In [ ]:
from sklearn.model_selection import train_test_split
X = df[feature_columns]
y = df.Outcome
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
####pure SVM classifier

from sklearn.svm import SVC
from sklearn.metrics import f1_score
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


F-measure: 0.6052631578947367


**SVM used in bagging to create diffetent models**

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import f1_score

svm = SVC(kernel='linear', random_state=42)
bagging = BaggingClassifier(base_estimator=svm, n_estimators=15, random_state=42)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


F-measure: 0.6184210526315789


**Initialize different classifiers***

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
svm = SVC(kernel='linear',random_state=42)
naive_bayes = GaussianNB()
decision_tree = DecisionTreeClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
base_estimators = [svm, naive_bayes, decision_tree, random_forest]
bagging = BaggingClassifier(base_estimator=None)
bagging.estimators_ = base_estimators
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


F-measure: 0.6666666666666666


**BOOSTING**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.7, random_state=42)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

F-measure: 0.6363636363636365


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.7, random_state=42)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

F-measure: 0.6503067484662576


In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


[LightGBM] [Info] Number of positive: 188, number of negative: 349
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000092 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 595
[LightGBM] [Info] Number of data points in the train set: 537, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.350093 -> initscore=-0.618630
[LightGBM] [Info] Start training from score -0.618630
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

**Stacking**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier


('rf', RandomForestClassifier(n_estimators=10, random_state=42)),


In [ ]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
              ('knn', KNeighborsClassifier(n_neighbors=5)),
              ('dt', DecisionTreeClassifier(random_state=42))]
meta_estimator = svm
stacking = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


F-measure: 0.6292134831460674


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import numpy as np
# Base classifiers
base_estimators = [
    RandomForestClassifier(n_estimators=10, random_state=42),
    KNeighborsClassifier(n_neighbors=5),
    DecisionTreeClassifier(random_state=42)
]

# Train base classifiers
predictions = []
for estimator in base_estimators:
    estimator.fit(X_train, y_train)
    preds = estimator.predict(X_train)
    predictions.append(preds)

# Convert predictions to features
X_train_stacked = np.column_stack(predictions)

# Neural network as the meta-estimator
meta_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_stacked.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train meta-estimator
meta_model.fit(X_train_stacked, y_train, epochs=10, batch_size=32)

# Make predictions on test data
predictions_test = []
for estimator in base_estimators:
    preds_test = estimator.predict(X_test)
    predictions_test.append(preds_test)

X_test_stacked = np.column_stack(predictions_test)
y_pred_probs = meta_model.predict(X_test_stacked)
y_pred = (y_pred_probs > 0.5).astype(int)


from sklearn.metrics import f1_score
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

Epoch 1/10
17/17 [==============================] - 1s 2ms/step - loss: 0.6454 - accuracy: 0.9516
Epoch 2/10
17/17 [==============================] - 0s 2ms/step - loss: 0.5579 - accuracy: 0.9963
Epoch 3/10
17/17 [==============================] - 0s 3ms/step - loss: 0.4675 - accuracy: 0.9963
Epoch 4/10
17/17 [==============================] - 0s 2ms/step - loss: 0.3825 - accuracy: 0.9963
Epoch 5/10
17/17 [==============================] - 0s 2ms/step - loss: 0.3028 - accuracy: 0.9963
Epoch 6/10
17/17 [==============================] - 0s 2ms/step - loss: 0.2272 - accuracy: 0.9981
Epoch 7/10
17/17 [==============================] - 0s 3ms/step - loss: 0.1600 - accuracy: 0.9981
Epoch 8/10
17/17 [==============================] - 0s 3ms/step - loss: 0.1067 - accuracy: 1.0000
Epoch 9/10
17/17 [==============================] - 0s 3ms/step - loss: 0.0710 - accuracy: 1.0000
Epoch 10/10
8/8 [==============================] - 0s 2ms/step
F-measure: 0.5942857142857143
